In [19]:
import os 
try:
    from os import scandir, walk
except ImportError:
    from scandir import scandir, walk
from nbformat import v4 as nbf
import re
def get_m_files(d="."):
    exts=["m"]
    for root, dirs, files in walk(d, topdown=False):
        for name in files:
            if "code_to_convert" in name:
                continue
            if os.path.splitext(name)[1].strip('.') in exts:
                yield os.path.abspath(os.path.join(root, name))

In [20]:
for m_file in get_m_files("m-scripts"):
    break

In [21]:
m_file

'/home/jafrey/Downloads/UMich_Controls_Tutorials_Python/m-scripts/Introduction/Introduction_SystemModeling.m'

In [22]:
example, section = os.path.splitext(os.path.basename(m_file))[0].split("_")

In [23]:
example

'Introduction'

In [24]:
section

'SystemModeling'

In [36]:
with open (m_file, "r") as fid:
    lines=fid.readlines()

In [37]:
lines = [line.strip() for line in lines]

In [40]:
first_header = True
for line in lines:
    if line.startswith("%% "):
        if first_header:
            line = line.replace("%%", "#")
        else:
            line = line.replace("%%", "##")
    elif line.is("%"):
        line="\n"
    elif line.startswith("% "):
        

    print(line)

# Introduction: System Modeling
%
% The first step in the control design process is to develop appropriate
% mathematical models of the system derived either from physical laws or
% experimental data. In this section, we introduce the state-space and
% transfer function representations of dynamic systems. We then review some
% basic approaches to modeling mechanical and electrical systems and show
% how to enter these models into MATLAB for further analysis.
%
% Key MATLAB commands used in this tutorial are:
% <http://www.mathworks.com/help/toolbox/control/ref/ss.html |ss|> ,
% <http://www.mathworks.com/help/toolbox/control/ref/tf.html |tf|>
%
# Dynamic Systems
%
% *Dynamic systems* are systems that change or evolve in time according to
% a fixed rule. For many physical systems, this rule can be stated as a set
% of first-order differential equations:
%
% $$
% \dot{\mathbf{x}} = \frac{d\mathbf{x}}{dt} = \mathbf{f}\left( \mathbf{x}(t), \mathbf{u}(t), t \right)
% $$
%
% In the above equa

In [29]:
line.strip()

'%% Introduction: System Modeling'

In [ ]:
   
    last_mtplt=None
    for line in lines:
        mtplt = matlab_to_python_line_transmogrification(line)
        if last_mtplt is None:
            running_block=""           
            notebook_cells=list()
        else:
            if mtplt.cell_type != last_mtplt.cell_type:
                if last_mtplt.cell_type == 3:
                    cell_tmp = nbf.new_markdown_cell(running_block.strip())
                if last_mtplt.cell_type == 2:
                    cell_tmp = nbf.new_code_cell(running_block.strip())
                notebook_cells.append(cell_tmp)
                running_block=str_tmp=str(mtplt)
                last_mtplt = mtplt
                continue
        str_tmp=str(mtplt)
        if len(str_tmp) == 0:
            continue
        if running_block.endswith("\n"):
            running_block=(running_block+str_tmp)
        else:
            running_block=(running_block+" "+str_tmp)
        last_mtplt = mtplt
        
    ## Start Cells
    nb["cells"].append(nbf.new_code_cell(r"""%pylab 
%matplotlib inline
import sympy
from control.matlab import *
from sympy import Poly
from sympy.abc import s, z
from IPython.display import display,Latex"""))
    nb["cells"].append(nbf.new_markdown_cell(r"""<div id="toc"></div>"""))
        
    ## Body cells
    for notebook_cell in notebook_cells:
        cleaned_cell=cell_cleaner_upperer(notebook_cell)
        nb["cells"].append(cleaned_cell)
        
    ## End Cells
    nb["cells"].append(nbf.new_code_cell(r"""%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')"""))

    nikola_metadata=dict()
    nikola_metadatas=dict()
    nikola_metadata["author"]=""
    nikola_metadata["category"]=example
    nikola_metadata["date"]=""
    nikola_metadata["description"]=""
    nikola_metadata["link"]=""
    nikola_metadata["slug"]=slug
    nikola_metadata["tags"]=section
    nikola_metadata["title"]=title
    nikola_metadata["type"]=""
    nikola_metadatas["nikola"]=nikola_metadata
    nb["metadata"]=nikola_metadatas
    return nb

In [ ]:
from nbconvert.preprocessors.execute import ExecutePreprocessor

In [2]:
   
class matlab_to_python_line_transmogrification(object):
    HEADER=0
    COMMENT=1
    CODE=2
    MARKDOWN=3
    """ trans·mog·ri·fied
        trans·mog·ri·fy·ing
        transitive verb
        to change or alter greatly and often with grotesque or humorous effect
    """
    def __init__(self, line):
        self.line = line
        if line.startswith("%%"):
            self.type=matlab_to_python_line_transmogrification.HEADER
            self.cell_type = matlab_to_python_line_transmogrification.MARKDOWN
        elif line.startswith("%"):
            self.type=matlab_to_python_line_transmogrification.COMMENT
            self.cell_type = matlab_to_python_line_transmogrification.MARKDOWN
        else:
            self.type=matlab_to_python_line_transmogrification.CODE
            self.cell_type = matlab_to_python_line_transmogrification.CODE
            
    def __repr__(self):
        return "mtplt({})".format(self.line)

    def __str__(self):
        line=self.line.strip()
        line=line.strip("% ")
        if len(line)==0:
            return ""
        if self.type == matlab_to_python_line_transmogrification.HEADER:    
            return "\n\n# {}\n\n".format(line)
        if self.type == matlab_to_python_line_transmogrification.COMMENT:          
            badwords=["Key MATLAB commands", "www.mathworks.com"]
            for badword in badwords:
                if badword in line:
                    return ""
            if line.startswith("* "):
                line="\n"+line
            prefix=""
            postfix=""
            return "{}{}{}".format(prefix,line,postfix)
        if self.type == matlab_to_python_line_transmogrification.CODE:
            line=self.line
            # print("C. "+line)
            return "{}".format(line)
        return ""

# http://fma.wikia.com/wiki/Homunculus
class Homunculus(object):
    def __init__(self, matlab_line, variable):
        self.matlab_line=matlab_line.strip()
        self.python_line=matlab_line
        self.variable = variable
        self.alchemy()
        
    def alchemy(self):
        value = self.matlab_line
        value=value.replace("\t","")
        value=value.replace("\n","")
        for q in range(10): 
            value=value.replace("  "," ")
        values = [V.replace(" ",",") for V in value.split("; ")]
        if len(values)==1:
            values = values[0].split(";")
        
        pythonic = "numpy.array([["+("],[".join(values)+"]])")
        
        regex=r"""numpy.array\(\[\[([\d.]+):([\d.]+):([\d.]+)\]\]\)"""
        match = re.compile(regex).findall(pythonic)
        if len(match) == 1:
            start, step, end = match[0]
            pythonic = "numpy.arange({start},{end},{step})".format(start=start,end=end,step=step)
        regex=r"""numpy.array\(\[\[([\d.]+):([\d.]+)\]\]\)"""
        match = re.compile(regex).findall(pythonic)
        if len(match) == 1:
            start, end = match[0]
            pythonic = "numpy.arange({start},{end})".format(start=start,end=end,step=step)
        try:
            eval(pythonic)
        except NameError:
            pass
        except:
            if False:
                raise
            else:
                pythonic = "# "+pythonic
        self.python_line = pythonic
        
    def __repr__(self):
        return self.variable+" = "+self.python_line
    
def matlab_to_python_code_converter(code_block):
    assignments = re.compile("([\w_]+) ?= ? \[([^]]+)\]").findall(code_block)
    assignment_blocks=list()
    for assignment in assignments:
        variable=assignment[0]
        sin=Homunculus(assignment[1],assignment[0])
        assignment_blocks.append(str(sin))
    code_blocks=list()
    for line in code_block.split("\n"):
        try:
            eval(line)
            code_blocks.append(line)
        except SyntaxError:
            code_blocks.append("# "+line)
        except NameError:
            code_blocks.append("# "+line)
        except TypeError:
            if line.startswith("title"):
                code_blocks.append("plt."+line)  
                continue
            if line.startswith("ylabel"):
                code_blocks.append("plt."+line)  
                continue
            if line.startswith("xlabel"):
                code_blocks.append("plt."+line)
                continue
            if line.startswith("plot"):
                code_blocks.append("plt."+line)
                continue
            code_blocks.append("# "+line)
        except:
            raise Exception(line)
    return "\n".join(assignment_blocks)+"\n"+"\n".join(code_blocks)

def cell_cleaner_upperer(notebook_cell):
    source = notebook_cell["source"]
    if notebook_cell["cell_type"] is "markdown":
        source=markdown_cleaner_upperer(source)
    if notebook_cell["cell_type"] is "code":
        source=matlab_to_python_code_converter(source)
        #print(source)
    #print(source, end="")
    notebook_cell["source"] = source
    return notebook_cell
def markdown_cleaner_upperer(markdown_block):
     # Non inline latex
    markdown_block=re.sub(r"\$\$([^$]+)\$\$",r"\n\n$$\n\1\n$$\n\n",markdown_block)
    # Inline Latex
    markdown_block=re.sub(r" \$([^$]+)\$ ",r" \\\\(\1\\\\) ",markdown_block)
    # Links to other
    markdown_block=re.sub(r"< \?example=([\w]+)&section=([\w]+) ([^>]+)>",
                  r"[\3](/examples/\1/\1-\2.ipynb)",markdown_block)
    markdown_block=re.sub(r"< ?\?aux=([\w _]+) ([^<]+)>",
                  r"[\2](/examples/\1.ipynb)",markdown_block)
    markdown_block=re.sub(r"<<([^>]+)>>",r"\n\n![image](http://ctms.engin.umich.edu/CTMS/\1)\n\n",markdown_block)
    markdown_block=re.sub(r"\*([\w -]+)\*",r"**\1**",markdown_block)
    return markdown_block

pp = ExecutePreprocessor()
pp.timeout = 30  # seconds
pp.interrupt_on_timeout = True

for m in get_m_files():
    ipynb = os.path.splitext(m)[0]+"_.ipynb"
    nb = convert_m_script_to_ipython_notebook(m)
    with open(ipynb, 'w') as f:
        if True:
            try:
                nb_executed, resources = pp.preprocess(nb, resources={})
                print(nbf.writes(nb_executed), file=f)
            except:
                print(nbf.writes(nb), file=f)
        else:
            print(nbf.writes(nb), file=f)

In [3]:
# Sweet zombie jesus, it ran.